# カテゴリカル分布
goodかbadの二択問題で、goodの確率が $\mu$ であるとき、goodである確率とbadである確率をまとめて書くと
<span id="eq1">
\begin{equation}
P(t)=\mu^t(1-\mu)^{1-t}
\end{equation}
</span>
と書けた。

次に、三択以上の確率分布を考えよう。サイコロの目のように、出目が6通りあるとする。このときの出目を、以下のようなone-hot表現で表すことにする。
$$\boldsymbol{x}=(0, 0, 1, 0, 0, 0)^\top$$

このとき、one-hot表現の出目ベクトル $\boldsymbol{x}$ の確率は
<span id="eq2">
\begin{equation}
P(\boldsymbol{x}|\boldsymbol{\mu})=\prod_{m=1}^6 \mu_m^{x_m}
\end{equation}
</span>
となる。 $\boldsymbol{\mu}=(\mu_1,\mu_2,\cdots)$ はパラメータベクトルである。

$K$ 回サイコロを振ったデータをまとめて $\boldsymbol{\mathsf{X}}=\begin{pmatrix}
\boldsymbol{x}_1^\top \\
\boldsymbol{x}_2^\top \\
\vdots
\end{pmatrix}$ と書くことにすると、このモデルから $\boldsymbol{\mathsf{X}}$ が発生する同時確率は

<span id="eq3">
\begin{equation}
P(\boldsymbol{\mathsf{X}}|\boldsymbol{\mu})=\prod_{k=1}^K\prod_{m=1}^6 \mu_m^{x_{km}}
\end{equation}
</span>

この同時確率を $\boldsymbol{\mu}$ の関数とみなし、これを尤度関数という。よって対数尤度関数は
<span id="eq4">
\begin{equation}
\ln L(\boldsymbol{\mu})=\ln P(\boldsymbol{\mathsf{X}}|\boldsymbol{\mu})=\sum_{k=1}^K\sum_{m=1}^6 x_{km}\ln\mu_m
\end{equation}
</span>
この関数を最大にするような $\boldsymbol{\mu}$ を求めればよい。

ところで、$m$ の目が $c_m$ 回出たとすると、 $c_m=\sum_{k=1}^K x_{km}$ だから
<span id="eq5">
\begin{equation}
\ln L(\boldsymbol{\mu})=\sum_{m=1}^6 c_m\ln\mu_m
\end{equation}
</span>
と書いてもよい。

## 勾配法によるパラメータ推定
尤度を最大にする $\boldsymbol{\mu}$ は解析的に求まり、
<span id="eq-ml">
\begin{equation}
\hat{\mu}_m=\frac{c_m}{K}
\end{equation}
</span>
である([課題](#kadai2))。わざわざパラメータ推定を勾配法で行う必要は全くないのだが、今回は練習のために敢えて勾配法で求めてみる。

6通りの出目を持つサイコロを100回振った結果から、パラメータ(=出目の確率)を推定しよう。まず、$\boldsymbol{\mu}$ の真値を決めておく。全部等確率だとおもしろくないので、2の目の確率を0.5, そのほかを0.1とする。

In [1]:
import numpy as np

mu_true = [0.1, 0.5, 0.1, 0.1, 0.1, 0.1]
c = np.random.multinomial(100, mu_true)
c

array([14, 52,  7, 11,  8,  8])

次に、[式(2)](#eq3)の確率分布にしたがって100回サイコロを振る。

In [2]:
xk = np.random.choice(6,100,p=mu_true)
xk+1

array([1, 5, 6, 1, 2, 2, 2, 1, 3, 2, 2, 2, 2, 3, 2, 2, 2, 4, 6, 6, 5, 5,
       2, 6, 1, 3, 2, 2, 3, 2, 2, 5, 5, 1, 2, 2, 6, 5, 2, 6, 2, 2, 2, 2,
       4, 6, 2, 2, 4, 6, 5, 2, 2, 4, 5, 5, 2, 4, 5, 2, 2, 3, 2, 2, 2, 6,
       4, 2, 5, 2, 4, 2, 2, 2, 6, 3, 1, 6, 2, 3, 6, 6, 2, 6, 6, 2, 4, 2,
       3, 2, 1, 2, 2, 6, 3, 2, 3, 4, 2, 3])

出目の回数は bincount 関数で数えられる。

In [3]:
c = np.bincount(xk)
c

array([ 7, 46, 11,  9, 11, 16])

xkをone-hot表現に変換しよう。

In [4]:
X = np.identity(6)[xk]
X[0:10,:]

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

[式(4)](#eq4)にしたがい、Xがデータとして与えられたときの対数尤度関数は次のように定義できる。

In [5]:
def categorical_likelihood(mu):
    return np.sum(X * np.log(mu))

one-hotベクトルでなく、出目の回数cがデータとして与えられた場合の対数尤度関数は、[式(5)](#eq5)より次のように定義できる。

In [6]:
def multinomial_likelihood(mu):
    return np.sum(c * np.log(mu))

これらが同じ結果を与えることを確認しておこう。適当な $\mu$ に対して

In [7]:
mu = np.array([0.2,0.1,0.2,0.1,0.3,0.1])
print(categorical_likelihood(mu))
print(multinomial_likelihood(mu))

-205.69712487397635
-205.69712487397632


同じ結果になった。以下ではデータとしてcを与えることにする。

In [8]:
# 「ゼロから作るDeep Learning」 p.104

def numerical_gradient(f, x):
    h = 1.0e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad

In [9]:
# 「ゼロから作るDeep Learning」 p.107

def gradient_descent(f, init_x, lr=0.001, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

損失関数は負の対数尤度で定義し、categorical_entropy_lossという名前を付けておく。

ここで1つコツがある。 $\mu_1,\mu_2,\cdots$ の総和は1でなければならないが、モデルパラメータにそのような制約を入れるのは少し厄介である。そこで、パラメータは和が1にならなくてもよいこととし、それらを $\mu^\text{raw}_{1}, \mu^\text{raw}_{2},\cdots$ と書く(つまり $\mu^\text{raw}_{1}$ 等は確率ではない)。そして、確率は
$$\mu_m = \frac{\mu^\text{raw}_{m}}{\sum_{m=1}^6\mu^\text{raw}_{m}}$$
のようにして計算する。

In [10]:
def categorical_entropy_loss(mu_raw):
    mu = mu_raw / np.sum(mu_raw)
    return -multinomial_likelihood(mu)

では、パラメータを推定してみよう。

In [11]:
mu_raw = gradient_descent(categorical_entropy_loss, np.random.rand(6),lr=0.01)
mu = mu_raw / np.sum(mu_raw)
mu

array([0.07      , 0.46000009, 0.10999999, 0.08999999, 0.10999999,
       0.15999995])

# 多クラスロジスティック回帰
ロジスティック回帰を多クラス問題に拡張しよう。多クラスロジスティック回帰モデルは、線形予測子 $z_m$ および softmax 関数 $\text{softmax}(z_m)=\frac{\exp(z_m)}{\sum_j \exp(z_j)}$ により以下のように書くことができる ($m$ はクラス番号)。
<span id="multiclasslogistic">
\begin{align*}
z_m&=\boldsymbol{w_m}^\top\boldsymbol{x} = \boldsymbol{x}^\top\boldsymbol{w_m}\\
y_m&=\text{softmax}(z_m)=\frac{\exp(z_m)}{\sum_j \exp(z_j)}
\end{align*}
</span>

ここで、$\boldsymbol{w}_m$ はクラス $m$ の確率を求めるためのパラメータで、(説明変数の数+1)次元のベクトルである（線形回帰やロジスティック回帰と同じ）。このベクトルをクラス数分横に並べたのがパラメータ行列 $\boldsymbol{W}=(\boldsymbol{w}_1 \boldsymbol{w}_2 \cdots \boldsymbol{w}_M)$ である。

予測すべきクラスが $M$ 個あるので、線形予測子は $z_1, z_2, \cdots z_M$ の $M$ 個ある。これらをひとまとめにすると
\begin{aligned}
\boldsymbol{z}&=(z_1 z_2 \cdots z_M) \\
&=\boldsymbol{x}^\top\boldsymbol{W}
\end{aligned}

$k$番目のデータの説明変数を $\boldsymbol{x}_k$, 目的変数を $\boldsymbol{t}_k$ とする。ただし $\boldsymbol{t}_k=(t_{k1} t_{k2} \cdots t_{kM})^\top$ は$k$番目のデータの正解クラスを表す one-hot 表現である。

データセット $\boldsymbol{\mathsf{X}}=\begin{pmatrix}
\boldsymbol{x}_1^\top \\
\boldsymbol{x}_2^\top \\
\vdots
\end{pmatrix}$,
$\boldsymbol{\mathsf{T}}=\begin{pmatrix}
\boldsymbol{t}_1^\top \\
\boldsymbol{t}_2^\top \\
\vdots
\end{pmatrix}$ に対し、 
$k$番目のデータの線形予測子ベクトルは $\boldsymbol{z}_k=\boldsymbol{x}_k^\top\boldsymbol{W}$
となる。よって、これを $K$ 個のデータ全部に対してまとめた行列 $\boldsymbol{Z}$ は

\begin{equation}
\boldsymbol{\mathsf{Z}}=\begin{pmatrix}
\boldsymbol{z}_1 \\
\boldsymbol{z}_2 \\
\vdots
\end{pmatrix}
=\boldsymbol{X}^\top\boldsymbol{W}
\end{equation}
なる $K\times M$行列である。

$k$番目のデータの線形予測子ベクトル $\boldsymbol{z}_k$ から得られる各クラスの確率(予測値) をまとめて $\boldsymbol{y}_k=(y_{k1} y_{k2} \cdots y_{kM})$ で表す。
$y_{km}=\text{softmax}(z_{km})$ であるが、ここで $M$ 次元ベクトル $\boldsymbol{z}_k$ を引数とし、 $M$ 個の予測値 $y_{k1} y_{k2} \cdots y_{kM}$ からなるベクトル $\boldsymbol{y}_k$ を返すベクトル版の softmax 関数をあらためて定義する。
\begin{equation}
\boldsymbol{y}_k=\text{softmax}(\boldsymbol{z}_k)
\end{equation}

したがって、これを $K$ 個のデータ全部に対してまとめた行列
$\boldsymbol{\mathsf{Y}}=\begin{pmatrix}
\boldsymbol{y}_1 \\
\boldsymbol{y}_2 \\
\vdots
\end{pmatrix}$
は、まず $\boldsymbol{Z}=\boldsymbol{X}^\top\boldsymbol{W}$
を求め、次にその各行の softmax を求め、最後にそれを縦に並べて作ればよい。

ロジスティック回帰で仮定した確率の[式(1)](#eq1)を[式(2)](#eq2)に置き換える。説明変数 $\boldsymbol{x}_k$ によって多クラスロジスティック回帰モデルで予測されるクラス $\boldsymbol{t}_k$ の確率は
\begin{equation}
P(\boldsymbol{t}_k|\boldsymbol{w})=\prod_m y_{km}^{t_{km}}
\end{equation}

$\boldsymbol{x}_1, \boldsymbol{x}_2 \cdots$ のそれぞれが $\boldsymbol{t}_1, \boldsymbol{t}_2 \cdots$ になる同時確率は、それぞれの確率の積になるから
\begin{equation}
P(\boldsymbol{\mathsf{T}}|\boldsymbol{w})=\prod_k \prod_m y_{km}^{t_{km}}
\end{equation}

よって、交差エントロピー（=負の対数尤度）は
<span id="eq-ent">
\begin{equation}
E=-\ln P(\boldsymbol{\mathsf{T}}|\boldsymbol{w})=-\sum_k \sum_m t_{km} \ln y_{km}
\end{equation}
</span>
となる。これは、行列 $\boldsymbol{\mathsf{T}}$ の各要素 $t_{km}$ と、行列 $\boldsymbol{\mathsf{Y}}$ の各要素 $y_{km}$ の対数の積を要素ごとに計算し、全ての要素に対して和を取る計算に相当する。

## iris (アヤメの判別)
ロジスティック回帰を用いてアヤメの花の寸法から3種類のアヤメを判別する問題を解いてみよう。irisデータセットについては各自調べておいてください。

In [12]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
T = np.identity(3)[iris.target]

説明変数は4つ。

In [13]:
X[0:10,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

目的変数tは3種類のアヤメの種類のone-hot表現。

In [14]:
T[0:5,:] , T[145:,:]

(array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]))

In [15]:
X_train = X[0::2] # 偶数番目
X_test = X[1::2] # 奇数番目
T_train = T[0::2]
T_test = T[1::2]

softmax関数を定義する。クラス数を $M$ とすると、引数は $M$ 次元のベクトル、戻り値は $M$ 次元のベクトルになる。

In [16]:
# ベクトル版 softmax 関数
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

この softmax 関数でも用は足りるが、データ全体を一挙に処理できる行列版の softmax 関数を作っておくと便利でしかも高速である。これは、行列の各行を1つのデータだとみなして softmax を実行し、結果の横ベクトルをデータ数だけ縦に並べた行列を返す関数である。

In [17]:
# 行列版 softmax 関数
def softmax(X):
    return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)

[式(11)](#eq-ent)に従って損失関数を定義する。

In [18]:
def categorical_cross_entropy_loss(W):
    Z = X_train_a.dot(W) # (KxD) x (DxM)= (KxM)
    Y = softmax(Z)
    return -np.sum(T_train * np.log(Y))

パラメータ $\boldsymbol{W}=(\boldsymbol{w}_1 \boldsymbol{w}_2 \boldsymbol{w}_3)$ は今度は行列になるので、多次元配列に対応したバージョンの numerical_gradient を使う。([ここ](https://github.com/oreilly-japan/deep-learning-from-scratch)から入手できます)

In [19]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

In [20]:
X_train_a = np.column_stack((np.ones(X_train.shape[0]),X_train))
X_test_a = np.column_stack((np.ones(X_test.shape[0]),X_test))

In [21]:
W=gradient_descent(categorical_cross_entropy_loss, np.random.rand(5,3), lr=0.001,step_num=2000)

テストデータに対してロジスティック回帰モデルによる予測を行う。各データが各クラスの花である確率は次のように計算できる。

In [22]:
Y=softmax(X_test_a.dot(W))
Y

array([[9.74095230e-01, 2.59047659e-02, 3.96506782e-09],
       [9.75138532e-01, 2.48614573e-02, 1.04707831e-08],
       [9.93547039e-01, 6.45296026e-03, 2.76321659e-10],
       [9.87897651e-01, 1.21023482e-02, 9.01165670e-10],
       [9.75390348e-01, 2.46096496e-02, 2.88701765e-09],
       [9.84761639e-01, 1.52383582e-02, 2.87125635e-09],
       [9.89445617e-01, 1.05543806e-02, 1.94996174e-09],
       [9.98919080e-01, 1.08092023e-03, 3.53659676e-12],
       [9.92314101e-01, 7.68589833e-03, 3.77256535e-10],
       [9.95444927e-01, 4.55507320e-03, 1.76323022e-10],
       [9.93780102e-01, 6.21989715e-03, 4.50954043e-10],
       [9.69307837e-01, 3.06921475e-02, 1.56338415e-08],
       [9.56580582e-01, 4.34194078e-02, 1.03151762e-08],
       [9.90410919e-01, 9.58908057e-03, 3.31748775e-10],
       [9.74851664e-01, 2.51483266e-02, 9.37913269e-09],
       [9.85886910e-01, 1.41130887e-02, 7.95834279e-10],
       [9.98762744e-01, 1.23725608e-03, 3.14928354e-12],
       [9.90615891e-01, 9.38410

$\boldsymbol{y}=(y_1 y_2 y_3)$ の中で確率が最大のクラスが認識結果となる。

In [23]:
result = np.array([np.argmax(y) for y in Y])
result

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2])

正解精度を求める。

In [24]:
np.sum(np.equal(result, [np.argmax(t) for t in T_test])) / T_test.shape[0]

0.9466666666666667

In [25]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='newton-cg',multi_class='multinomial')
logreg.fit(iris.data,iris.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
-np.sum(T_train * logreg.predict_log_proba(X_train))

8.868623402058041

In [27]:
-np.sum(T_train * np.log(softmax(X_train_a.dot(W))))

7.069775925779655

# 課題
1. [softmax関数の式](#multiclasslogistic)で、クラスの数を2とするとロジスティック関数と同じになることを示せ。
1. <span id="kadai2"> [カテゴリカル分布の最尤推定式](#eq-ml)を証明せよ。</span>(ヒント: この問題は制約付き最大化問題になるのでLagrangeの未定乗数法を使うと解ける。)
1. 上で未定義のまま残しておいた categorical_cross_entropy_loss 関数を完成させ、アヤメの判別を実行せよ。